# Nettoyage des données V2

In [1]:
import s3fs
import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)

### Récupération des données

In [3]:
from scripts.importation import process_all_years_s3

In [4]:
# Connexion au bucket S3
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

ROOT_S3_PATH = "mvallat/diffusion/projet"

In [5]:
# Importation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_S3, 'rb') as file_in:
    data_brevets = pd.read_parquet(file_in)

data_brevets.head(10)

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,applicant_1_postcode,applicant_1_country,applicant_2_orgname,applicant_2_address-1,applicant_2_city,applicant_2_postcode,applicant_2_country,applicant_3_orgname,applicant_3_address-1,applicant_3_city,applicant_3_postcode,applicant_3_country,inventor_1_last-name,inventor_1_first-name,inventor_1_address-1,inventor_1_city,inventor_1_postcode,inventor_1_country,inventor_2_last-name,inventor_2_first-name,inventor_2_address-1,inventor_2_city,inventor_2_postcode,inventor_2_country,inventor_3_last-name,inventor_3_first-name,inventor_3_address-1,inventor_3_city,inventor_3_postcode,inventor_3_country,agent_1_orgname,agent_1_address-1,agent_1_city,agent_1_postcode,agent_1_country,agent_2_orgname,agent_2_address-1,agent_2_city,agent_2_postcode,agent_2_country,agent_3_orgname,agent_3_address-1,agent_3_city,agent_3_postcode,agent_3_country,owner_1_last-name,owner_1_first-name,owner_1_address-1,owner_1_city,owner_1_postcode,owner_1_country,owner_2_last-name,owner_2_first-name,owner_2_address-1,owner_2_city,owner_2_postcode,owner_2_country,owner_3_last-name,owner_3_first-name,owner_3_address-1,owner_3_city,owner_3_postcode,owner_3_country,classification_1_text,classification_2_text,classification_3_text,abstract,citation_1_type,citation_1_text,citation_1_country,citation_1_doc-number,citation_1_date,citation_2_type,citation_2_text,citation_2_country,citation_2_doc-number,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,FAURECIA INTERIEUR INDUSTRIE None,"2, RUE HENNAPE",NANTERRE,92000,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,BERTINELLI,RAPHAEL,20 RUE D'ECHAVANNE,CHENEBIER,70400,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,"2, PLACE D'ESTIENNE D'ORVES ,",PARIS,75009,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,F16B 5/07 20060101AFI20150702BHFR ...,F16B 5/08 20060101ALI20150702BHFR ...,B60J 5/00 20060101ALI20150702BHFR ...,La présente invention concerne un ensemble (10...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,IFP ENERGIES NOUVELLES None,1 ET 4 AVENUE DE BOIS-PREAU,RUEIL MALMAISON CEDEX,92852,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,MILOSAVLJEVIC,MISA,17 RUE CLAUDE BENARD,ERAGNY SUR OISE,95610,FR,ZITO,GIANLUCA,12 RUE DE PONTOISE,PARIS,75005,FR,NA,NA,NA,NA,NA,NA,NA,95 RUE D'AMSTERDAM,PARIS CEDEX 8,75378,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,B60W 20/00 20060101AFI20150630BHFR ...,B60W 10/08 20060101ALI20150630BHFR ...,B60W 10/115 20120101ALI20150630BHFR ...,Dans un véhicule hybride comprenant un moteur ...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
2,3038409,A1,FR,PUBDEM,FR,3038409,20170106,2017-01,Brevet,PROCEDE DE CONCEPTION ASSISTEE PAR ORDINATEUR ...,CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE None,3 RUE MICHEL ANGE,PARIS,75016,FR,UNIVERSITE D'AIX-MARSEILLE None,58 BOULEVARD CHARLES LIVON,MARSEILLE CEDEX 7,13284,FR,ECOLE CENTRALE DE MARSEILLE None,TECHNOPOLE DE CHATEAU GOMBERT\n38 RUE FREDERIC...,MARSEILLE,13013,FR,DUPONT,GUILLAUME,28 RUE DE VERDUN,MARSEILLE,13005,FR,ENOCH,STEFAN,15 RUE DE L'ESCALET\nLA COLLINE N°17,MARSEILLE,13013,FR,MOLIN,BERNARD,20 CHEMIN DE LA RESQUILLETTE,VENELLES,13770,FR,NA,310 AVENUE BERTHELOT,LYON,69008,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,G06F 17/50 20060101AFI20150701BHFR ...,G06F 17/11 20060101ALI20150701BHFR ...,E02B 3/06 20060

### Observation des variables

#### **1) Type de brevet**

In [6]:
# Colonne "kind"
data_brevets["kind"].value_counts()

kind
A1    107749
A3      4441
NA       102
B1         3
Name: count, dtype: int64

Le document contient trois types de brevets :

- **A1** : demande de brevet  
- **A3** : demande de certificat d'utilité  
- **B1** : brevet délivré  


**Concernant les types B1 (n = 3)** : cette base recensant les demandes de brevet, elle n'est pas censée contenir un brevet B1.  
Probablement dû à une erreur de classification, nous pouvons donc retirer ces observations.

In [7]:
# Suppression des B1
data_brevets_clean = data_brevets[data_brevets["kind"] != "B1"]

# Vérification qu'on a supprimé 3 observations
data_brevets_clean.shape[0]


112292

**Concernant les types A3 (n = 4 441)** : les certificats d'utilité sont des certificats délivrés par l'INPI qui protègent l'innovation, tout comme les brevets. En revanche la durée de protection est raccourcie (10 ans contre 20 ans) et la procédure d'examen de la demande est allégée. Pour simplifier l'analyse en la rendant plus homogène, nous ne gardons pas ces demandes A3 dans notre base.

In [8]:
# Suppression des A3
data_brevets_clean = data_brevets_clean[data_brevets_clean["kind"] != "A3"]

# Vérification : suppression de 4 441 observations (112,292 - 4,441 = 107,851)
data_brevets_clean.shape[0]

107851

**Concernant les NA (n = 102)** : plus d'observations sont nécessaires pour décider de la manière de gérer ces NA.


In [9]:
# Observation des NA
brevets_na = data_brevets[data_brevets["kind"] == "NA"]
brevets_na


,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,applicant_1_postcode,applicant_1_country,applicant_2_orgname,applicant_2_address-1,applicant_2_city,applicant_2_postcode,applicant_2_country,applicant_3_orgname,applicant_3_address-1,applicant_3_city,applicant_3_postcode,applicant_3_country,inventor_1_last-name,inventor_1_first-name,inventor_1_address-1,inventor_1_city,inventor_1_postcode,inventor_1_country,inventor_2_last-name,inventor_2_first-name,inventor_2_address-1,inventor_2_city,inventor_2_postcode,inventor_2_country,inventor_3_last-name,inventor_3_first-name,inventor_3_address-1,inventor_3_city,inventor_3_postcode,inventor_3_country,agent_1_orgname,agent_1_address-1,agent_1_city,agent_1_postcode,agent_1_country,agent_2_orgname,agent_2_address-1,agent_2_city,agent_2_postcode,agent_2_country,agent_3_orgname,agent_3_address-1,agent_3_city,agent_3_postcode,agent_3_country,owner_1_last-name,owner_1_first-name,owner_1_address-1,owner_1_city,owner_1_postcode,owner_1_country,owner_2_last-name,owner_2_first-name,owner_2_address-1,owner_2_city,owner_2_postcode,owner_2_country,owner_3_last-name,owner_3_first-name,owner_3_address-1,owner_3_city,owner_3_postcode,owner_3_country,classification_1_text,classification_2_text,classification_3_text,abstract,citation_1_type,citation_1_text,citation_1_country,citation_1_doc-number,citation_1_date,citation_2_type,citation_2_text,citation_2_country,citation_2_doc-number,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
285,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
606,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
831,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
1153,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
1543,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28236,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,N

In [10]:
# Vérification du nombre d'informations réellement présentes
brevets_na["country"].nunique()
brevets_na["invention-title"].nunique()

2

On peut constater que seules deux observations contiennent des informations. Nous pouvons donc supprimer toutes les observations qui ne contiennent que des NA.

In [11]:
# Suppression des observations 100% NA
data_brevets_clean = data_brevets_clean[data_brevets_clean["kind"] != "NA"]

#### **2) Numéro du document**

In [12]:
# Vérification que doc_number est unique

nb_number = data_brevets_clean["doc-number"].nunique()
print("Nombre de doc-number uniques :", nb_number)

nb_number2 = data_brevets_clean["publication_doc-number"].nunique()
print("Nombre de publication_doc-number uniques :", nb_number2)

nb_observations = data_brevets_clean.shape[0]
print("Nombre total d'observations :", nb_observations)


Nombre de doc-number uniques : 107748
Nombre de publication_doc-number uniques : 107748
Nombre total d'observations : 107749


Nous pouvons constater qu'il y a une observation qui n'a pas un numéro unique.

In [13]:
doublons = data_brevets_clean[data_brevets_clean.duplicated(subset=["doc-number"], keep=False)]
print(doublons)

      doc-number kind country  status publication_country  \
12476    3050837   A1      FR  PUBDEM                  FR   
58919    3050837   A1      FR     NEW                  FR   

      publication_doc-number publication_date publication_bopinum  \
12476                3050837         20171103             2017-44   
58919                3050837         20171103             2017-44   

      publication_nature                                    invention-title  \
12476             Brevet  DISPOSITIF D'INFORMATION INERTIELLE, SAGITTALE...   
58919  Demande française  DISPOSITIF D'INFORMATION INERTIELLE, SAGITTALE...   

                  applicant_1_orgname      applicant_1_address-1  \
12476  BOARDING RING READING SAS None  661 CHE DU CHATEAU VALLON   
58919       BOARDING RING READING SAS  661 CHE DU CHATEAU VALLON   

      applicant_1_city applicant_1_postcode applicant_1_country  \
12476        OLLIOULES                83190                  FR   
58919        OLLIOULES         

#### **3) Statut**

La documentation de l'INPI nous apprend que la variable "status" est utilisée pour indiquer quel est l'évènement qui a entraîné la présence du brevet ou du CCP dans le fichier de la semaine. Les 

In [29]:
# Colonne "status"
data_brevets_clean["publication_nature"].value_counts()

publication_nature
Demande française    79625
Brevet               28124
Name: count, dtype: int64

In [14]:
data_brevets_clean.shape[0]

107749

#### **4) Abstract**
Les abstracts sont les résumés des brevets déposés. Il s'agit pour nous de vérifier qu'ils contiennent bien du texte d'une part, et qu'il n'y a pas de doublons d'autre part.

In [40]:
counts = data_brevets_clean["abstract"].value_counts()
print(counts[counts > 1])

abstract
NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

On peut constater qu'il y a 1166 abstracts en double, dont 339 NA. 

In [41]:
data_brevets_NA = data_brevets_clean[
    data_brevets_clean["abstract"] == "NA"
]
data_brevets_NA

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,applicant_1_postcode,applicant_1_country,applicant_2_orgname,applicant_2_address-1,applicant_2_city,applicant_2_postcode,applicant_2_country,applicant_3_orgname,applicant_3_address-1,applicant_3_city,applicant_3_postcode,applicant_3_country,inventor_1_last-name,inventor_1_first-name,inventor_1_address-1,inventor_1_city,inventor_1_postcode,inventor_1_country,inventor_2_last-name,inventor_2_first-name,inventor_2_address-1,inventor_2_city,inventor_2_postcode,inventor_2_country,inventor_3_last-name,inventor_3_first-name,inventor_3_address-1,inventor_3_city,inventor_3_postcode,inventor_3_country,agent_1_orgname,agent_1_address-1,agent_1_city,agent_1_postcode,agent_1_country,agent_2_orgname,agent_2_address-1,agent_2_city,agent_2_postcode,agent_2_country,agent_3_orgname,agent_3_address-1,agent_3_city,agent_3_postcode,agent_3_country,owner_1_last-name,owner_1_first-name,owner_1_address-1,owner_1_city,owner_1_postcode,owner_1_country,owner_2_last-name,owner_2_first-name,owner_2_address-1,owner_2_city,owner_2_postcode,owner_2_country,owner_3_last-name,owner_3_first-name,owner_3_address-1,owner_3_city,owner_3_postcode,owner_3_country,classification_1_text,classification_2_text,classification_3_text,abstract,citation_1_type,citation_1_text,citation_1_country,citation_1_doc-number,citation_1_date,citation_2_type,citation_2_text,citation_2_country,citation_2_doc-number,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
26036,3064421,A1,FR,PUBDEM,FR,3064421,20180928,2018-39,Brevet,ROTOR POUR MOTEUR OU GENERATEUR ELECTROMAGNETI...,WHYLOT SAS None,PARC D'ACTIVITES QUERCYPOLE 2,CAMBES,46100,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,RAVAUD,ROMAIN,8 LES CANAVALS HAUTS,LABASTIDE MURAT,46240,FR,MAYEUR,LOIC,LA CROIX DU FABRE,SAINT SENTIN,12700,FR,NA,NA,NA,NA,NA,NA,NA,PARC D'ACTIVITES QUERCYPOLE 2,CAMBES,46100,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,H02K 1/27 20060101AFI20170918BHFR ...,H02K 1/06 20060101ALI20170918BHFR ...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2018,None,None,None,None,None,None
26188,3064439,A1,FR,PUBDEM,FR,3064439,20180928,2018-39,Brevet,DISPOSITIF DE RETROVISION VIDEO POUR VEHICULE ...,DAV None,76 RUE AUGUSTE PERRET - ZI EUROPARC,CRETEIL CEDEX,94046,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,DABIC,STEPHANIE,C/O DAV\n76 RUE AUGUSTE PERRET - ZI EUROPARC,CRETEIL CEDEX,94046,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,SERVICE PROPRIETE INTELLECTUELLE\n76 RUE AUGUS...,CRETEIL CEDEX,94046,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,H04N 5/232 20060101AFI20170322BHFR ...,B60R 1/062 20060101ALI20170322BHFR ...,G06T 7/00 20170101ALI20170322BHFR ...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2018,None,None,None,None,None,None
30080,3068197,A1,FR,NEW,FR,3068197,20181228,2018-52,Demande française,CARTOUCHE DE CARACTERISATION ELECTRONIQUE DES ...,SWISSLOG FRANCE,28 QUAI GALLIENI,SURESNES CEDEX,92156,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NDUHURA MUNGA,JUSTIN,28 QUAI GALLIENI,SURESNES,92156,FR,COCHINI,BASTIEN,28 QUAI GALLIENI,SURESNES,92156,FR,BENAMARA,MOHAMED,28 QUAI GALLIENI,SURESNES,92156,FR,NA,24 RUE DE PARIS,HERBLAY,95220,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,28 QUAI GALLIENI,SURESNES CEDEX,92156,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,H05K 1/02 20060101AFI20180313BH ...,F25D 3/08 20060101ALI20180313BH ...,G06Q 50/30 20120101ALI20180313BH ...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2018,FR,1770683,20170627,None,None,None
30673,3068791,A1,FR,NEW,FR,3068791,20190111,2019-02,Demande française,ANALYSE APPROFONDIE DE L'ORTEQUE DE VERRE OPHT...,GIROD DENIS PAUL,71 BOULEVARD DE STRAS

- **Gestion des NA** : En observant la base de données, on constate que ces abstract vides sont parfois associés à des demandes incomplètes, et parfois le reste de la demande semble complète. Après quelques recherches sur internet, il nous semble que l'explication la plus probable est que ces demandes soient déposées sans leur abstract, qui est fourni plus tard à l'INPI (l'abstract étant obligatoire pour la demande). Pour se concentrer sur les demandes complètes uniquement, nous décidons donc de supprimer ces observations, qui restent en faible nombre par rapport à nos plus de 100,000 observations.

In [49]:
# Drop des NA
data_brevets_clean = data_brevets_clean[
    data_brevets_clean["abstract"] != "NA"
]

# Vérification 
nb_na = (data_brevets_clean["abstract"] == "NA").sum()
print("Nombre de NA après le drop :", nb_na)

Nombre de NA après le drop : 0


- **Gestion des abstracts en double** : 

In [62]:
# compte des abstracts strictement identiques
duplicates_exact = data_brevets_clean[data_brevets_clean.duplicated(subset="abstract", keep=False)]
print("Nombre d'observations avec des doublons :", len(duplicates_exact))

Nombre d'observations avec des doublons : 2691


On commence par parcourir les observations avec des doublons pour se donner une idée de la raison de l'existence de ces doublons.

In [61]:
duplicates_exact.sort_values("abstract")

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,applicant_1_postcode,applicant_1_country,applicant_2_orgname,applicant_2_address-1,applicant_2_city,applicant_2_postcode,applicant_2_country,applicant_3_orgname,applicant_3_address-1,applicant_3_city,applicant_3_postcode,applicant_3_country,inventor_1_last-name,inventor_1_first-name,inventor_1_address-1,inventor_1_city,inventor_1_postcode,inventor_1_country,inventor_2_last-name,inventor_2_first-name,inventor_2_address-1,inventor_2_city,inventor_2_postcode,inventor_2_country,inventor_3_last-name,inventor_3_first-name,inventor_3_address-1,inventor_3_city,inventor_3_postcode,inventor_3_country,agent_1_orgname,agent_1_address-1,agent_1_city,agent_1_postcode,agent_1_country,agent_2_orgname,agent_2_address-1,agent_2_city,agent_2_postcode,agent_2_country,agent_3_orgname,agent_3_address-1,agent_3_city,agent_3_postcode,agent_3_country,owner_1_last-name,owner_1_first-name,owner_1_address-1,owner_1_city,owner_1_postcode,owner_1_country,owner_2_last-name,owner_2_first-name,owner_2_address-1,owner_2_city,owner_2_postcode,owner_2_country,owner_3_last-name,owner_3_first-name,owner_3_address-1,owner_3_city,owner_3_postcode,owner_3_country,classification_1_text,classification_2_text,classification_3_text,abstract,citation_1_type,citation_1_text,citation_1_country,citation_1_doc-number,citation_1_date,citation_2_type,citation_2_text,citation_2_country,citation_2_doc-number,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
14977,3053138,A1,FR,PUBDEM,FR,3053138,2017-12-29,2017-52,Brevet,PROCEDE D'EDITION AUTOMATISEE D'UN BLOC D'UN T...,LUCKY CART None,159 RUE MONTMARTRE,PARIS,75002,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,CHAPPE,OLIVIER,31 Chemin des Grandes Combes 69360 TERNAY,TERNAY,69360,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,"29, RUE DE LISBONNE",PARIS,75008,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,G06F 3/0484 20130101AFI20160622BHFR ...,G06F 3/0482 20130101ALI20160622BHFR ...,NA,\n,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
74543,3112481,A1,FR,NEW,FR,3112481,2022-01-21,2022-03,Demande française,COMPOSITION POUR TEINDRE DES FIBRES DE KÉRATIN...,L'OREAL,"14, rue Royale",PARIS,75008,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,MA,Jingmiao,L'OREAL R & D JAPAN KSP R&D B1213 / 3-2-1 SAK...,KAWASAKI,NA,JP,JIANG,YUEHUANG,L'OREAL R&I JAPAN KSP R&D B1213 / 3-2-1 SAKAD...,KAWASAKI,NA,JP,LIU,ZHIBING,L'OREAL R&I JAPAN KSP R&D B1213 / 3-2-1 SAKAD...,KAWASAKI,NA,JP,Lavoix,"62, rue de Bonnel",LYON CEDEX 03,69448,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,"14, rue Royale",PARIS,75008,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,A61K 8/49 20191216AFI20201021BHFR ...,A61K 8/34 20191216ALI20201021BHFR ...,A61Q 5/10 20191216ALI20201021BHFR ...,\n \n \n Composit...,patcit,CN 101 961 302 B (GUANGZHOU YOUNGRACE\nDAILY C...,CN,CN-101961302-B,20130529,patcit,CN 107 349 131 A (ZHONGSHAN JIALI FINE\nCHEMIC...,CN,CN-107349131-A,20171117,patcit,GB 1 065 223 A (OREAL)\n12 avril 1967 (1967-04...,GB,GB-1065223-A,19670412,2022,FR,2007612,20200720,20210611,20220801,20220121
65623,3103688,A1,FR,NEW,FR,3103688,2021-06-04,2021-22,Demande française,Dispositif destiné à assurer la connexion de s...,READYNOVATION,9 rue Volta Chez Monsieur Geoffroy GLORIEUX,PARIS,75003,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,LE BRETON,Yann,5 rue villa d'Aurion,ROSNY SOUS BOIS,93110,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,9 rue Volta Chez Monsieur Geoffroy GLORIEUX,PARIS,75003,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,A45C 13/30 20191216AFI20200720BHFR ...,A45C 3/06 20191216ALI20200720BHFR ...,NA,\n \n \n Disposit...,NA,NA,NA,NA,

**1. Cas des doublons parfaits :**

On veut d'abord vérifier si il existe des observations qui ont exactement les mêmes valeurs pour toutes les variables sauf pour le numéro de dossier.

In [ ]:
# Vérification existance doublon parfait
cols_comparaison = [
    c for c in data_brevets_clean.columns
    if c not in ["doc_number", "publication_doc_number"]
]


n_duplicates = data_brevets_clean.duplicated(
    subset=cols_comparaison
).sum()
print(n_duplicates)

Il n'y a donc aucun doublon parfait ! Toutes les demandes avec le même abstract ...

**2. Cas des doublons avec des dates différentes :**

Nous émettons l'hypothèse que les observations avec des NA en double correspondent à la même demande de brevet déposée plusieurs fois afin d'ajouter une correction d'une version à une autre. Ainsi, nous choisissons de considérer que la demande la plus récente entre les doublons est la demande la plus aboutie, et donc la seule que nous garderons. 

In [ ]:
# Garder la demande la plus ancienne parmi les doublons

## Passer publication_date au format date
data_brevets_clean["publication_date"] = pd.to_datetime(
    data_brevets_clean["publication_date"],
    format="%Y%m%d",
    errors="coerce"
)
## Trouver la date la plus ancienne par abstract
max_date_per_abstract = (
    duplicates_exact
    .groupby("abstract")["publication_date"]
    .transform("max")
)

## Garder les doublons les plus récents
duplicates_keep = duplicates_exact[
    duplicates_exact["publication_date"] == max_date_per_abstract
]

## Reconstruire la base
data_brevets_clean2 = pd.concat(
    [
        data_brevets_clean[~data_brevets_clean.index.isin(duplicates_exact.index)],
        duplicates_keep
    ],
    axis=0
).sort_index()

## Compter le nouveau nombre de brevets
duplicates_exact2 = data_brevets_clean2[data_brevets_clean2.duplicated(subset="abstract", keep=False)]
print("Nouveau nombre d'observations avec des doublons :", len(duplicates_exact2))


Nombre d'observations avec des doublons : 1872


0


In [44]:
data_brevets_clean.groupby("abstract").filter(lambda x: len(x) > 1)

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,applicant_1_postcode,applicant_1_country,applicant_2_orgname,applicant_2_address-1,applicant_2_city,applicant_2_postcode,applicant_2_country,applicant_3_orgname,applicant_3_address-1,applicant_3_city,applicant_3_postcode,applicant_3_country,inventor_1_last-name,inventor_1_first-name,inventor_1_address-1,inventor_1_city,inventor_1_postcode,inventor_1_country,inventor_2_last-name,inventor_2_first-name,inventor_2_address-1,inventor_2_city,inventor_2_postcode,inventor_2_country,inventor_3_last-name,inventor_3_first-name,inventor_3_address-1,inventor_3_city,inventor_3_postcode,inventor_3_country,agent_1_orgname,agent_1_address-1,agent_1_city,agent_1_postcode,agent_1_country,agent_2_orgname,agent_2_address-1,agent_2_city,agent_2_postcode,agent_2_country,agent_3_orgname,agent_3_address-1,agent_3_city,agent_3_postcode,agent_3_country,owner_1_last-name,owner_1_first-name,owner_1_address-1,owner_1_city,owner_1_postcode,owner_1_country,owner_2_last-name,owner_2_first-name,owner_2_address-1,owner_2_city,owner_2_postcode,owner_2_country,owner_3_last-name,owner_3_first-name,owner_3_address-1,owner_3_city,owner_3_postcode,owner_3_country,classification_1_text,classification_2_text,classification_3_text,abstract,citation_1_type,citation_1_text,citation_1_country,citation_1_doc-number,citation_1_date,citation_2_type,citation_2_text,citation_2_country,citation_2_doc-number,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
48,3038286,A1,FR,PUBDEM,FR,3038286,20170106,2017-01,Brevet,ENSEMBLE COMPRENANT UN GARDE-BOUE POUR VEHICUL...,PEUGEOT CITROEN AUTOMOBILES SA None,ROUTE DE GISY,VELIZY VILLACOUBLAY,78140,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,BERTH,STEPHANE,21 RUE DES PIERRELAIS,CHATILLON,92320,FR,GUILBAUT,MARC,31 RUE DU PRESIDENT KENNEDY,ALFORTVILLE,94140,FR,NA,NA,NA,NA,NA,NA,NA,SERVICE PROPRIETE INDUSTRIELLE\n18 RUE DES FAU...,LA GARENNE COLOMBES,92250,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,B62D 25/16 20060101AFI20150703BHFR ...,NA,NA,Ensemble (2) comprenant :,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
76,3038249,A1,FR,PUBDEM,FR,3038249,20170106,2017-01,Brevet,VITRAGE CHAUFFANT A FEUILLE DE VERRE EXTERIEUR...,SAINT-GOBAIN GLASS FRANCE None,18 AVENUE D'ALSACE,COURBEVOIE,92400,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,TONDU,THOMAS,27 RUE DES VERGERS,SULLY,45600,FR,CHAUSSADE,PIERRE,8 ALLEE DU VERGER,ORLEANS,45000,FR,LEGOIS,VINCENT,8 CHEMIN DU PONT CHALUMEAU \nLA PRIEUREE,GERMIGNY DES PRES,45110,FR,NA,DEPARTEMENT PROPRIETE INDUSTRIELLE - 39 QUAI L...,AUBERVILLIERS,93300,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,B32B 17/10 20060101AFI20150702BHFR ...,H05B 3/84 20060101ALI20150702BHFR ...,B64C 1/14 20060101ALI20150702BHFR ...,L'invention concerne,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
84,3038325,A1,FR,PUBDEM,FR,3038325,20170106,2017-01,Brevet,PROCEDE DE CRYOCONSERVATION DE CELLULES A VISE...,LABORATOIRE FRANCAIS DU FRACTIONNEMENT ET DES ...,ZA DE COURTABOEUF 3 AV DES TROPIQUES,LES ULIS,91940,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,DE LARICHAUDY,JOFFREY,38 RUE SAVIER,MALAKOFF,92240,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,66 RUE DE LA CHAUSSEE D ANTIN,PARIS CEDEX 09,75440,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,C12N 5/02 20060101AFI20150630BHFR ...,A01N 1/00 20060101ALI20150630BHFR ...,NA,La présente invention se rapporte à une compos...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
131,3038279,A1,FR,PUBDEM,FR,3038279,20170106,2017-01,Brevet,PROCE

In [45]:
data_brevets_clean["abstract"].nunique()


105885

### Tri des variables pertinentes

In [17]:
data_brevets_light = data_brevets[['publication_date', 'invention-title', 'classification_1_text', 'abstract']]
data_brevets_clean.head(5)


,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,...,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,...,NA,NA,NA,2017,None,None,None,None,None,None
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,...,NA,NA,NA,2017,None,None,None,None,None,None
2,3038409,A1,FR,PUBDEM,FR,3038409,20170106,2017-01,Brevet,PROCEDE DE CONCEPTION ASSISTEE PAR ORDINATEUR ...,...,NA,NA,NA,2017,None,None,None,None,None,None
3,3038425,A1,FR,PUBDEM,FR,3038425,20170106,2017-01,Brevet,DOCUMENT ELECTRONIQUE TEL QU'UNE CARTE A PUCE ...,...,NA,NA,NA,2017,None,None,None,None,None,None
4,3038316,A1,FR,PUBDEM,FR,3038316,20170106,2017-01,Brevet,PROCEDE DE SYNTHESE EN CONTINU D'UN ELASTOMERE...,...,NA,NA,NA,2017,None,None,None,None,None,None
